<a href="https://colab.research.google.com/github/Peter-obi/JAX/blob/main/Vmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import jax
import jax.numpy as jnp

In [3]:
def dot(v1, v2):
  return jnp.vdot(v1, v2) #use vdot() to calculate the dot product

In [4]:
dot(jnp.array([1., 1., 1.]), jnp.array([1., 2., -1]))  #calculates th edot product between 2 vectors

Array(2., dtype=float32)

In [6]:
rng_key = jax.random.PRNGKey(42) #generates a random number generator key

In [8]:
vs = jax.random.normal(rng_key, shape =(20, 3)) #generate a two-dimensional array of random numbers

In [9]:
v1s = vs[:10, :] #split array into two parts; the first 10 go the first list and the second 10 go to the second list
v2s = vs[10:, :]